In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage
import argparse
import cv2
import time
import matplotlib.image as mpimg
import scipy.ndimage as ndi
from PIL import Image, ImageOps
import os
import pprint
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops
import imageio as iio
import glob
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
import sys
import scipy.special


In [3]:
class Gaussian:
#---Initializing variables---
    def __init__(self, A, x0, y0, sigmax, sigmay, tL, tR, var, mu, std):
        self.A = A
        self.x0 = x0
        self.y0 = y0
        self.sigmax = sigmax
        self.sigmay = sigmay
        self.mu = mu
        self.std = std
        self.tL = tL
        self.tR = tR
        self.var = var
        self.X = np.arange(-tL,tR,var)
        self.Y = np.arange(-tL,tR,var)
        self.X, self.Y = np.meshgrid(self.X, self.Y)
        self.path = os.getcwd()

####################################################################################

#---generate data---
    def generate(self):
        shift = [0, .5, -.5, 1, -1]
        self.G1 = self.A * np.exp(-((((self.X + shift[0])**2) / (2 * (self.sigmax**2))) 
                                    + (((self.Y + shift[0])**2) / (2 * (self.sigmay**2)))))
        self.G2 = self.A * np.exp(-((((self.X + shift[1])**2) / (2 * (self.sigmax**2))) 
                                    + (((self.Y + shift[1])**2) / (2 * (self.sigmay**2)))))
        self.G3 = self.A * np.exp(-((((self.X + shift[2])**2) / (2 * (self.sigmax**2))) 
                                    + (((self.Y + shift[2])**2) / (2 * (self.sigmay**2)))))
        self.G4 = self.A * np.exp(-((((self.X + shift[3])**2) / (2 * (self.sigmax**2))) 
                                    + (((self.Y + shift[3])**2) / (2 * (self.sigmay**2)))))
        self.G5 = self.A * np.exp(-((((self.X + shift[4])**2) / (2 * (self.sigmax**2))) 
                                    + (((self.Y + shift[4])**2) / (2 * (self.sigmay**2)))))
        self.gaus = [self.G1, self.G2, self.G3, self.G4, self.G5]
        return
    def generate_multi(self):
        self.gaus = []
        shift = [0, .5, -.5, 1, -1]
        for i in range(0,5):
            self.G1 = self.A * np.exp(-((((self.X - (self.x0 + .75) + shift[i])**2) / (2 * (self.sigmax**2))) 
                                        + (((self.Y - (self.y0 - 1) + shift[i])**2) / (2 * (self.sigmay**2)))))
            self.G2 = self.A * np.exp(-((((self.X - (self.x0 - .75) + shift[i])**2) / (2 * (self.sigmax**2))) 
                                        + (((self.Y - (self.y0 + .75) + shift[i])**2) / (2 * (self.sigmay**2)))))
            self.G3 = self.A * np.exp(-((((self.X - (self.x0 - .9) + shift[i])**2) / (2 * (self.sigmax**2))) 
                                        + (((self.Y - (self.y0 - .5) + shift[i])**2) / (2 * (self.sigmay**2)))))
            self.G = 10*self.G1 + 3*self.G2 + 5*self.G3
            print(shift)
            self.gaus.append(self.G)
        print(len(self.gaus))
        return
    def airy_disc(self):
        self.X = np.linspace(-10,10,500)
        self.Y = np.linspace(-10,10,500)
        self.X, self.Y = np.meshgrid(self.X, self.Y)
        for i in range(0,5):
            shift = np.random.uniform(-5,5)
            self.G1 = 3 * np.exp(-((((self.X + shift)**2) / (2 * (self.sigmax**2))) + (((self.Y + shift)**2) / (2 * (self.sigmay**2)))))
            r = np.sqrt((self.X + shift)**2 + (self.Y + shift)**2)
            AD =( 1 * (scipy.special.j1(r) / r)**2 ) + np.exp(-1.5*r)*np.cos(2*np.pi*r + np.pi/8)
            airy = AD + self.G1
            maxval = np.amax(airy)
            plt.contourf(self.X, self.Y, airy, 500, cmap="gray", vmax = (maxval * (1 - (i*.185))))
            plt.colorbar()
            plt.xlabel('x')
            plt.ylabel('y')
            plt.title('Gaussian Beam with obscured Airy disc')
            plt.show()
        return
    def generate_no_noise(self):
        self.noise = [np.zeros((np.size(self.X,0),np.size(self.Y,0))),
                      np.zeros((np.size(self.X,0),np.size(self.Y,0))),
                      np.zeros((np.size(self.X,0),np.size(self.Y,0))), 
                      np.zeros((np.size(self.X,0),np.size(self.Y,0))),
                      np.zeros((np.size(self.X,0),np.size(self.Y,0)))]
        return
    
    def generate_noise(self):
        self.noise1 = np.random.normal(self.mu, self.std, size = self.gaus[0].shape)
        self.noise2 = np.random.normal(self.mu, self.std, size = self.gaus[0].shape)
        self.noise3 = np.random.normal(self.mu, self.std, size = self.gaus[0].shape)
        self.noise4 = np.random.normal(self.mu, self.std, size = self.gaus[0].shape)
        self.noise5 = np.random.normal(self.mu, self.std, size = self.gaus[0].shape)
        self.noise = [self.noise1, self.noise2, self.noise3, self.noise4, self.noise5]
        return

    def final_sim_beam(self):
        self.gaus = [self.gaus[0] + self.noise[0], self.gaus[1] + self.noise[1], self.gaus[2] 
                     + self.noise[2], self.gaus[3] + self.noise[3], self.gaus[4] + self.noise[4]]
#         print("Generated multiple gaussians")
#         self.multi_name = "Gmulti_PM"
        print("Generated final sim beam profile")
        self.multi_name = "Gfinal_sim_beam"
        self.image = str(self.multi_name) + ".png"
        return

#---import images---
    def importim(self):
        path = os.getcwd()
#         print(path)
        subfolder = '\lotti_images'    # Target directory/subfolder with single images
        os.chdir(path+subfolder)
        dir_list = os.listdir(path+subfolder)
#         print(dir_list)
        image_list = []
        for filename in dir_list:
            if filename.endswith(".png"):
                img = cv2.imread(filename, cv2.IMREAD_ANYDEPTH)
                cv2.imwrite(filename, img)
                image_list.append(filename)
#         print(image_list)
        self.image_list = image_list
        return
###################################################################################
#---where to send imported images---
    def senddataPM(self):
        PM_list = self.image_list
        self.call_CAC(PM_list)
        self.patternmatching(PM_list)
        return
    def senddataPI(self):
        PI_list = self.image_list
        self.pngPI(PI_list)
        return
    def senddataCOM(self):
        COM_list = self.image_list
        self.pngCOM(COM_list)
        return

####################################################################################

#---center based on simulated data---
    #---peak intensity---
    def simPI(self):
        print("Centering SIM images using Peak Intensity")
        maxval = np.amax(self.gaus)
        PI_list = []
        for i in range(0,5):
            gaus = self.gaus[i]
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_aspect('equal')
            PI_name = str(self.multi_name) + str(i)
            plt.contourf(self.X,self.Y,self.gaus[i], 500, cmap='gist_gray', vmax = (maxval * (1 - (i*.175))))
            plt.axis('off')
            plt.savefig(PI_name, bbox_inches = "tight", pad_inches=0)
            PI_png = PI_name + ".png"
            PI_list.append(PI_png)          
#             self.simcenterPI(gaus, PI_name)
        print(PI_list)
        for i in range(len(PI_list)):
            PI_name = str(self.multi_name) + str(i)
            img = cv2.imread(PI_list[i], cv2.IMREAD_GRAYSCALE)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(img)
            # Calculate the translation matrix
            tx = img.shape[1] // 2 - max_loc[0]
            ty = img.shape[0] // 2 - max_loc[1]
            M = np.float32([[1, 0, tx], [0, 1, ty]])
# Translate the image
            translated_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
# Save the translated image
            cv2.imwrite(PI_name + "_centered_PI.png", translated_img)
        return 

    #---center of mass---
    def simCOM(self):
        print("Centering SIM images using Center of Mass")
        maxval = np.amax(self.gaus)
        COM_list = []
        for i in range(0,5):
            gaus = self.gaus[i]
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_aspect('equal')
            COM_name = str(self.multi_name) + str(i)
            plt.contourf(self.X,self.Y,self.gaus[i], 500, cmap='gist_gray', vmax = (maxval * (1 - (i*.175))))
            plt.axis('off')
            plt.savefig(COM_name, bbox_inches = "tight", pad_inches=0)
            COM_png = COM_name + ".png"
            COM_list.append(COM_png)          
#             self.simcenterPI(gaus, PI_name)
        print(COM_list)
        for i in range(len(COM_list)):
            COM_name = str(self.multi_name) + str(i)
            img = cv2.imread(COM_list[i], cv2.IMREAD_GRAYSCALE)
            M = cv2.moments(img)
# Calculate the x and y coordinates of the center of mass
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
# Print the coordinates of the center of mass
            # Calculate the translation matrix
            tx = img.shape[1] // 2 - cx
            ty = img.shape[0] // 2 - cy
            M = np.float32([[1, 0, tx], [0, 1, ty]])
# Translate the image
            translated_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    
# Save the translated image
            cv2.imwrite(str(self.multi_name) + str(i) + "_centered_COM.png", translated_img)
        return 
####################################################################################
    
#how do you want to center and plot the data? 
    def pngPI(self, PI_list):
        print("Centering PNG images using Peak Intensity")
        for i in range(len(PI_list)):
            PI_name = "PNG_beam" + str(i)
            img = cv2.imread(PI_list[i], cv2.IMREAD_ANYDEPTH)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(img)
            # Calculate the translation matrix
            tx = img.shape[1] // 2 - max_loc[0]
            ty = img.shape[0] // 2 - max_loc[1]
            M = np.float32([[1, 0, tx], [0, 1, ty]])
# Translate the image
            translated_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
# Save the translated image
            cv2.imwrite('C:\\Users\\AVazquez\\Desktop\\GitHub\\BELLA-Physics\\HDR-focus\\peak_intensity\\' 
                        + PI_name + "_centered_PI.png", translated_img)
        return
    def pngCOM(self, COM_list):
        for i in range(len(COM_list)):
            COM_name = "PNG_beam" + str(i)
            img = cv2.imread(COM_list[i], cv2.IMREAD_GRAYSCALE)
            M = cv2.moments(img)
# Calculate the x and y coordinates of the center of mass
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
# Print the coordinates of the center of mass
            # Calculate the translation matrix
            tx = img.shape[1] // 2 - cx
            ty = img.shape[0] // 2 - cy
            M = np.float32([[1, 0, tx], [0, 1, ty]])
# Translate the image
            translated_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
# Save the translated image
            cv2.imwrite('C:\\Users\\AVazquez\\Desktop\\GitHub\\BELLA-Physics\\HDR-focus\\center_of_mass\\'
                        + COM_name + "_centered_COM.png", translated_img)
        pass
####################################################################################


#---create 5 different images to pattern match--
    def simPM(self):
        maxval = np.amax(self.gaus)
        PM_list = []
        for i in range(0,5):
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.set_aspect('equal')
            PM_name = str(self.multi_name) + str(i)
            plt.contourf(self.X,self.Y,self.gaus[i], 500, cmap='gist_gray', vmax = (maxval * (1 - (i*.175))))
#             plt.title(str(self.multi_name) + str(i))
            plt.axis('off')
            plt.savefig(PM_name, bbox_inches = "tight")
            PM_png = PM_name + ".png"
            PM_list.append(PM_png)
#             print("crop time")
        print(PM_list)
        self.call_CAC(PM_list)
        self.patternmatching(PM_list)
        return
    def patternmatching(self,PM_list):
        for i in range(len(PM_list)):
            image = cv2.imread(PM_list[i])
            center_check = PM_list[i]
            center_number = i
            mask = cv2.inRange(image, (0, 0, 0), (254, 254, 254))
            maskcheck = cv2.bitwise_and(image, image, mask=mask)
#             cv2.imshow('image with mask', maskcheck)
#             cv2.waitKey()
#             cv2.destroyAllWindows()
            template = cv2.imread(self.roi_name,cv2.COLOR_BGR2GRAY)
            mask = cv2.resize(mask, (template.shape[1], template.shape[0]))
            result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED, mask=mask)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
#             print(max_val)
            top_left = max_loc
            bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
#             print(top_left, bottom_right)
            w = template.shape[1]
            h = template.shape[0]
 # Get the center of the result
            result_center_x = int(max_loc[0] + template.shape[1] / 2)
            result_center_y = int(max_loc[1] + template.shape[0] / 2)
            # Translate the center of the result to the center of the image
            image_center_x = int(image.shape[1] / 2)
            image_center_y = int(image.shape[0] / 2)
            dx = int(image_center_x - result_center_x)
            dy = int(image_center_y - result_center_y)
# Translate the
#             print(center_coords)
            self.centerPM(center_check, top_left, bottom_right, dx,dy)
            cv2.rectangle(image, max_loc, (max_loc[0] + w, max_loc[1] + h), (0,255,255), 2)
#             cv2.imshow('PM result', image)
#             cv2.waitKey()
#             cv2.destroyAllWindows()
        return

#---crop image using cursor---
    def click_and_crop(self, event, x, y, flags, param):
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being
    # performed
        if event == cv2.EVENT_LBUTTONDOWN:
            self.refPt = [(x, y)]
            self.cropping = True
    # check to see if the left mouse button was released
        elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
            # the cropping operation is finished
            self.refPt.append((x, y))
            self.cropping = False
        # draw a rectangle around the region of interest
            cv2.rectangle(self.image, self.refPt[0], self.refPt[1], (0, 255, 0), 1)
            cv2.imshow("image", self.image)
            return
        
    def call_CAC(self, PM_list):
        self.refPt = []
        self.cropping = False
        PM_choose = input("Which file would you like to use? \n" + str(PM_list))
        self.image = cv2.imread(PM_choose)
        self.clone = self.image.copy()
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", self.click_and_crop)
# keep looping until the 'q' key is pressed
        while True:
    # display the image and wait for a keypress
            cv2.imshow("image", self.image)
            self.key = cv2.waitKey(1) & 0xFF
    # if the 'r' key is pressed, reset the cropping region
            if self.key == ord("r"):
                self.image = self.clone.copy()
    # if the 'c' key is pressed, break from the loop
            elif self.key == ord("c"):
                break
# if there are two reference points, then crop the region of interest
# from teh image and display it
        if len(self.refPt) == 2:
            self.roi = self.clone[self.refPt[0][1]:self.refPt[1][1], self.refPt[0][0]:self.refPt[1][0]]
            im = Image.fromarray(self.roi)
            print("this is the reference point for the green box\n", 
                  self.refPt[0][1], self.refPt[1][1], self.refPt[0][0],self.refPt[1][0])
            self.roi_name = "temp" + PM_choose
            im.save(self.roi_name)
#             cv2.imshow("ROI", self.roi)
            cv2.waitKey(0)
# close all open windows
        cv2.destroyAllWindows()
    def centerPM(self, image, top_left, bottom_right, dx,dy):
        cropped_images = []
        img = cv2.imread(image, cv2.IMREAD_ANYDEPTH)
#         cropped_image = img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        translated_image = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        cv2.imwrite("Cropped" + image, translated_image)
#         cropped_images.append(cropped_image)

        return

In [3]:
        
#         cropped_images = []
#         path = os.getcwd()
#         subfolder = '\Mini_best_fok'    # target directory/subfolder with single captures
#         os.chdir(path+subfolder)
#         imageformat = "png"     # image format
#         name = "Mini"   
#         im1 = np.asarray(Image.open("{0}_setting_1_1.{1}".format(name, imageformat)))
#         print(im1.mode)
#         LEFT = 700 + top_left[0]
#         TOP =  700 + top_left[1]
#         RIGHT = 1400 - bottom_right[0]
#         BOTTOM = 1400 - bottom_right[1]
#         cropped = im1.crop((LEFT,TOP,RIGHT,BOTTOM))
#         cropped.save("Cropped" + image)
'''
        cropped_images = []
        img = cv2.imread(image)
        cropped_image = img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        cv2.imwrite("Cropped" + image, cropped_image)
#         cropped_images.append(cropped_image)
'''

NameError: name 'gaussian' is not defined